# Project - Twitter US Airline Sentiment

### 1. Overview
In this section, we're gonne do basic steps to:
- Import the libraries
- Load the dataset
- Print shape of data
- Print data description

In [16]:
pip install contractions

     |████████████████████████████████| 321 kB 5.6 MB/s 
     |████████████████████████████████| 283 kB 38.3 MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.2-cp37-cp37m-linux_x86_64.whl size=85441 sha256=dd415bffd30c03eb64a3dca921206dfaa0217c405c12629d65e47c2426ddf281
  Stored in directory: /root/.cache/pip/wheels/25/19/a6/8f363d9939162782bb8439d886469756271abc01f76fbd790f
Successfully built pyahocorasick


In [34]:
import pandas as pd
import numpy as np
import contractions
import nltk
from bs4 import BeautifulSoup
import re, string, unicodedata

nltk.download('stopwords')                              # Download Stopwords.
nltk.download('punkt')
nltk.download('wordnet')

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [4]:
# Connect with Google Colab
from google.colab import drive
drive.mount('/content/drive/')

# Read data from Tweets.csv stored in Google Colab Notebooks
data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Tweets.csv')

Mounted at /content/drive/


In [5]:
# Print the the shape of data
data.shape

(14640, 15)

In [6]:
data.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


### 2. Understand the data columns

In [7]:
# Drop all other columns except “text” and “airline_sentiment”.
data = data[['text', 'airline_sentiment']]

In [8]:
# Print new shape of data
data.shape

(14640, 2)

In [11]:
# Checking missing data
data.isna().sum()

text                 0
airline_sentiment    0
dtype: int64

In [13]:
data['airline_sentiment'].unique()

array(['neutral', 'positive', 'negative'], dtype=object)

In [12]:
# Check the first 5 rows of data
pd.set_option('display.max_colwidth', None)
data.head()

,text,airline_sentiment
0,@VirginAmerica What @dhepburn said.,neutral
1,@VirginAmerica plus you've added commercials to the experience... tacky.,positive
2,@VirginAmerica I didn't today... Must mean I need to take another trip!,neutral
3,"@VirginAmerica it's really aggressive to blast obnoxious ""entertainment"" in your guests' faces &amp; they have little recourse",negative
4,@VirginAmerica and it's a really big bad thing about it,negative


### 3. Text preprocessing: Data Preparation
As raw texts are unstructured, we can never use them for the training purpose. Therefore, We follow the steps below to structurize our data
- HTML tag removal
- Replace contractions
- Tokenization
- Remove stopwords
- Remove the numbers
- Remove special characters and punctuations
- Remove Non ASCII 
- Conversion to lowercase
- Lemmatize or Stemming
- Make complete processed text using "Join"


In [19]:
# HTML tag removal
def strip_html(text):
  soup = BeautifulSoup(text, 'html.parser')
  return soup.get_text()

data['text'] = data['text'].apply(lambda x: strip_html(x))
data.head()

,text,airline_sentiment
0,@VirginAmerica What @dhepburn said.,neutral
1,@VirginAmerica plus you have added commercials to the experience... tacky.,positive
2,@VirginAmerica I did not today... Must mean I need to take another trip!,neutral
3,"@VirginAmerica it is really aggressive to blast obnoxious ""entertainment"" in your guests' faces & they have little recourse",negative
4,@VirginAmerica and it is a really big bad thing about it,negative


In [18]:
# Replace contractions
def replace_contractions(text):
  return contractions.fix(text)

data['text'] = data['text'].apply(lambda x: replace_contractions(x))
data.head()

,text,airline_sentiment
0,@VirginAmerica What @dhepburn said.,neutral
1,@VirginAmerica plus you have added commercials to the experience... tacky.,positive
2,@VirginAmerica I did not today... Must mean I need to take another trip!,neutral
3,"@VirginAmerica it is really aggressive to blast obnoxious ""entertainment"" in your guests' faces & they have little recourse",negative
4,@VirginAmerica and it is a really big bad thing about it,negative


In [20]:
# Remove the numbers
# Remove special characters and punctuations
def remove_num_punc_special_char(words):
  special_char = r'[^a-zA-Z0-9]'
  return [re.sub(special_char,'', word) for word in words]

data['text'] = data['text'].apply(lambda x: remove_num_punc_special_char(x))
data.head()

,text,airline_sentiment
14635,@AmericanAir thank you we got on a different flight to Chicago.,positive
14636,@AmericanAir leaving over minutes Late Flight. No warnings or communication until we were minutes Late Flight. that is called shitty customer svc,negative
14637,@AmericanAir Please bring American Airlines to #BlackBerry,neutral
14638,"@AmericanAir you have my money, you change my flight, and do not answer your phones! Any other suggestions so I can make my commitment??",negative
14639,@AmericanAir we have ppl so we need know how many seats are on the next flight. Plz put us on standby for people on the next flight?,neutral


In [23]:
# Tokenization of data
data['text'] = data.apply(lambda row: nltk.word_tokenize(row['text']), axis=1)
data.head()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


,text,airline_sentiment
0,"[@, VirginAmerica, What, @, dhepburn, said, .]",neutral
1,"[@, VirginAmerica, plus, you, have, added, commercials, to, the, experience, ..., tacky, .]",positive
2,"[@, VirginAmerica, I, did, not, today, ..., Must, mean, I, need, to, take, another, trip, !]",neutral
3,"[@, VirginAmerica, it, is, really, aggressive, to, blast, obnoxious, ``, entertainment, '', in, your, guests, ', faces, &, they, have, little, recourse]",negative
4,"[@, VirginAmerica, and, it, is, a, really, big, bad, thing, about, it]",negative


In [35]:
# Download and make a list of stopwords
stopwords = stopwords.words('english')
customlist = ['not', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn',
        "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',
        "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn',
        "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"]
stopwords = list(set(stopwords) - set(customlist))

# Remove stopwords
def remove_stopwords(words):
  return [word for word in words if word not in stopwords]

data['text'] = data.apply(lambda row: remove_stopwords(row['text']), axis=1)
data.head()

,text,airline_sentiment
0,"[@, VirginAmerica, What, @, dhepburn, said, .]",neutral
1,"[@, VirginAmerica, plus, added, commercials, experience, ..., tacky, .]",positive
2,"[@, VirginAmerica, I, not, today, ..., Must, mean, I, need, take, another, trip, !]",neutral
3,"[@, VirginAmerica, really, aggressive, blast, obnoxious, ``, entertainment, '', guests, ', faces, &, little, recourse]",negative
4,"[@, VirginAmerica, really, big, bad, thing]",negative


In [41]:
# Remove Non ASCII 
def remove_non_ascii(words):
  return [unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore') for word in words]

data['text'] = data['text'].apply(lambda x: remove_non_ascii(x))
data.head()

,text,airline_sentiment
0,"[, VirginAmerica, What, , dhepburn, said, ]",neutral
1,"[, VirginAmerica, plus, added, commercials, experience, , tacky, ]",positive
2,"[, VirginAmerica, I, not, today, , Must, mean, I, need, take, another, trip, ]",neutral
3,"[, VirginAmerica, really, aggressive, blast, obnoxious, , entertainment, , guests, , faces, , little, recourse]",negative
4,"[, VirginAmerica, really, big, bad, thing]",negative


In [42]:
# Conversion to lowercase
def to_lowercase(words):
  return [word.lower() for word in words]

data['text'] = data['text'].apply(lambda x: to_lowercase(x))
data.head()

,text,airline_sentiment
0,"[, virginamerica, what, , dhepburn, said, ]",neutral
1,"[, virginamerica, plus, added, commercials, experience, , tacky, ]",positive
2,"[, virginamerica, i, not, today, , must, mean, i, need, take, another, trip, ]",neutral
3,"[, virginamerica, really, aggressive, blast, obnoxious, , entertainment, , guests, , faces, , little, recourse]",negative
4,"[, virginamerica, really, big, bad, thing]",negative


In [43]:
# Lemmatize or Stemming
lemmatizer = WordNetLemmatizer()

def lemmatize_list(words):
  return [lemmatizer.lemmatize(word, pos='v') for word in words]

data['text'] = data['text'].apply(lambda x: lemmatize_list(x))
data.head()

,text,airline_sentiment
0,"[, virginamerica, what, , dhepburn, say, ]",neutral
1,"[, virginamerica, plus, add, commercials, experience, , tacky, ]",positive
2,"[, virginamerica, i, not, today, , must, mean, i, need, take, another, trip, ]",neutral
3,"[, virginamerica, really, aggressive, blast, obnoxious, , entertainment, , guests, , face, , little, recourse]",negative
4,"[, virginamerica, really, big, bad, thing]",negative


In [44]:
# Make complete processed text using "Join"
data['text'] = data['text'].apply(lambda x: ' '.join(x))
data.head()

,text,airline_sentiment
0,virginamerica what dhepburn say,neutral
1,virginamerica plus add commercials experience tacky,positive
2,virginamerica i not today must mean i need take another trip,neutral
3,virginamerica really aggressive blast obnoxious entertainment guests face little recourse,negative
4,virginamerica really big bad thing,negative


### 4. Vectorization
There are 2 popular techniques to vectorize text data
- Count Vectorizer
- Tf-Idf Vectorizer

### 5. Fit and evaluate the model

### 6. Summarize
We summarize our understanding of the application of various data pre-processing and vectorization technique, also the performance of our model